## First design bayesian network based on expert (google) knowledge

(image included in report)


In [5]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
import numpy as np
from pgmpy.inference import VariableElimination

# Define structure of network from image, randomly assign parameters based on the network


In [6]:
# Define the structure of the Bayesian Network
networkStructure = [
    # level 1
    ("IsSummer", "HasFoodPoisoning"),
    ("IsSummer", "HasHayFever"),
    ("IsSummer", "HasFlu"),
    # level 2
    ("HasFoodPoisoning", "HasGastricProblems"),
    ("HasFoodPoisoning", "Vomits"),
    ("HasHayFever", "Rash"),
    ("HasHayFever", "Coughs"),
    ("HasHayFever", "Fever"),
    ("HasFlu", "Coughs"),
    ("HasFlu", "Fever"),
    ("HasPneumonia", "Coughs"),
    ("HasPneumonia", "Fever"),
    ("HasPneumonia", "HasRespiratoryProblems"),
    ("HasPneumonia", "IsFatigued"),
    # level 2.5
    ("HasGastricProblems", "Vomits"),
    # level 3, skips vomits and rash cause no children
    ("Coughs", "IsFatigued"),
    ("Fever", "IsFatigued"),
]
random_model = BayesianModel(networkStructure)

# randomly assign parameters => listing all as [prob(0) , prob(1)] for each parent configuration
cpd_IsSummer = TabularCPD(variable="IsSummer", variable_card=2, values=[[0.8], [0.2]])
cpd_HasFoodPoisoning = TabularCPD(
    variable="HasFoodPoisoning",
    variable_card=2,
    values=[[0.5, 0.3], [0.5, 0.7]],
    evidence=["IsSummer"],
    evidence_card=[2],
)
cpd_HasHayFever = TabularCPD(
    variable="HasHayFever",
    variable_card=2,
    values=[[0.5, 0.3], [0.5, 0.7]],
    evidence=["IsSummer"],
    evidence_card=[2],
)
cpd_HasFlu = TabularCPD(
    variable="HasFlu",
    variable_card=2,
    values=[[0.5, 0.3], [0.5, 0.7]],
    evidence=["IsSummer"],
    evidence_card=[2],
)
cpd_HasPneumonia = TabularCPD(
    variable="HasPneumonia", variable_card=2, values=[[0.2], [0.8]]
)
cpd_HasGastricProblems = TabularCPD(
    variable="HasGastricProblems",
    variable_card=2,
    values=[[0.5, 0.3], [0.5, 0.7]],
    evidence=["HasFoodPoisoning"],
    evidence_card=[2],
)
cpd_Vomits = TabularCPD(
    variable="Vomits",
    variable_card=2,
    values=[[0.5, 0.3, 0.2, 0.1], [0.5, 0.7, 0.8, 0.9]],
    evidence=["HasFoodPoisoning", "HasGastricProblems"],
    evidence_card=[2, 2],
)
cpd_Rash = TabularCPD(
    variable="Rash",
    variable_card=2,
    values=[[0.5, 0.3], [0.5, 0.7]],
    evidence=["HasHayFever"],
    evidence_card=[2],
)
cpd_Coughs = TabularCPD(
    variable="Coughs",
    variable_card=2,
    values=[
        [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
        [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2],
    ],
    evidence=["HasHayFever", "HasFlu", "HasPneumonia"],
    evidence_card=[2, 2, 2],
)
cpd_Fever = TabularCPD(
    variable="Fever",
    variable_card=2,
    values=[
        [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
        [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2],
    ],
    evidence=["HasHayFever", "HasFlu", "HasPneumonia"],
    evidence_card=[2, 2, 2],
)
cpd_HasRespiratoryProblems = TabularCPD(
    variable="HasRespiratoryProblems",
    variable_card=2,
    values=[[0.5, 0.3], [0.5, 0.7]],
    evidence=["HasPneumonia"],
    evidence_card=[2],
)
cpd_IsFatigued = TabularCPD(
    variable="IsFatigued",
    variable_card=2,
    values=[
        [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
        [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2],
    ],
    evidence=["HasPneumonia", "Coughs", "Fever"],
    evidence_card=[2, 2, 2],
)


# Verify the model structure and CPDs
print("Edges in the Bayesian Network:")
for edge in random_model.edges():
    print(edge)

Edges in the Bayesian Network:
('IsSummer', 'HasFoodPoisoning')
('IsSummer', 'HasHayFever')
('IsSummer', 'HasFlu')
('HasFoodPoisoning', 'HasGastricProblems')
('HasFoodPoisoning', 'Vomits')
('HasHayFever', 'Rash')
('HasHayFever', 'Coughs')
('HasHayFever', 'Fever')
('HasFlu', 'Coughs')
('HasFlu', 'Fever')
('HasGastricProblems', 'Vomits')
('Coughs', 'IsFatigued')
('Fever', 'IsFatigued')
('HasPneumonia', 'Coughs')
('HasPneumonia', 'Fever')
('HasPneumonia', 'HasRespiratoryProblems')
('HasPneumonia', 'IsFatigued')


/usr/local/lib/python3.10/dist-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


### add the cpds to the model and test consistency


In [7]:
# add cpds to the model
random_model.add_cpds(
    cpd_IsSummer,
    cpd_HasFoodPoisoning,
    cpd_HasHayFever,
    cpd_HasFlu,
    cpd_HasPneumonia,
    cpd_HasGastricProblems,
    cpd_Vomits,
    cpd_Rash,
    cpd_Coughs,
    cpd_Fever,
    cpd_HasRespiratoryProblems,
    cpd_IsFatigued,
)
# Check if the model is consistent
assert random_model.check_model()

# use the random model to create a inference object to be able to make a full assignment and get prob for it


In [9]:
# get value for a full assignment

# Create an exact inference object
ranodm_inference = VariableElimination(random_model)


# calculate full asignment
random_probability = ranodm_inference.query(
    variables=[
        "IsSummer",
        "HasFlu",
        "HasFoodPoisoning",
        "HasHayFever",
        "HasPneumonia",
        "HasRespiratoryProblems",
        "HasGastricProblems",
        "Rash",
        "Coughs",
        "IsFatigued",
        "Vomits",
        "Fever",
    ]
)

value = random_probability.get_value(
    IsSummer=1,
    HasFlu=0,
    HasFoodPoisoning=0,
    HasHayFever=0,
    HasPneumonia=1,
    HasRespiratoryProblems=1,
    HasGastricProblems=0,
    Rash=1,
    Coughs=1,
    IsFatigued=0,
    Vomits=0,
    Fever=0,
)
print("probability of a full assignment - ", value)

probability of a full assignment -  4.2336e-05


# import dataset and visualize


In [10]:
# Read the dataset from medical/dataset.dat
import pandas as pd

# dataset is a file with a list of assigments for each 12 variables, but represented as a integer. one line is one assigment
dataset = pd.read_csv("dataset.dat", header=None)
# visualize the dataset
print(dataset.head())

     0
0    1
1  136
2    8
3    0
4  129


# calculate cpds from the dataset instead of the random model we did

(more description in code comments and report)


In [11]:
# iterate over the dataset and count the number of times a variable is true for each assignment to its parents( if any)
# and then normalize the counts using the total number of times the parents had that assignment
# easier way i found is to count both 0 and 1 in different variables and assign probability easily.


cpd_IsSummerTotal = [0]
cpd_IsNotSummerTotal = [0]

cpd_HasFoodPoisoningTotal = [0, 0]
cpd_NoFoodPoisoningTotal = [0, 0]

cpd_HasHayFeverTotal = [0, 0]
cpd_NoHayFeverTotal = [0, 0]

cpd_HasFluTotal = [0, 0]
cpd_NoFluTotal = [0, 0]

cpd_HasPneumoniaTotal = [0]
cpd_NoPneumoniaTotal = [0]

cpd_HasGastricProblemsTotal = [0, 0]
cpd_NoGastricProblemsTotal = [0, 0]

cpd_VomitsTotal = [0, 0, 0, 0]
cpd_NoVomitTotal = [0, 0, 0, 0]

cpd_RashTotal = [0, 0]
cpd_NoRashTotal = [0, 0]

cpd_CoughsTotal = [0, 0, 0, 0, 0, 0, 0, 0]
cpd_NoCoughsTotal = [0, 0, 0, 0, 0, 0, 0, 0]

cpd_FeverTotal = [0, 0, 0, 0, 0, 0, 0, 0]
cpd_NoFeverTotal = [0, 0, 0, 0, 0, 0, 0, 0]

cpd_HasRespiratoryProblemsTotal = [0, 0]
cpd_NoRespiratoryProblemsTotal = [0, 0]

cpd_IsFatiguedTotal = [0, 0, 0, 0, 0, 0, 0, 0]
cpd_NotFatiguedTotal = [0, 0, 0, 0, 0, 0, 0, 0]

indexes = {
    "IsSummer": 0,
    "HasFlu": 1,
    "HasFoodPoisoning": 2,
    "HasHayFever": 3,
    "HasPneumonia": 4,
    "HasRespiratoryProblems": 5,
    "HasGastricProblems": 6,
    "Rash": 7,
    "Coughs": 8,
    "IsFatigued": 9,
    "Vomits": 10,
    "Fever": 11,
}


def getValue(binary, index):
    return int(binary[index])


def getNoValue(binary, index):
    return 1 - int(binary[index])


for _, row in dataset.iterrows():
    value = row.values[0]
    # convert the integer to a binary string
    binary = "{0:012b}".format(value)
    # get the values for each variable
    # no parents
    cpd_IsSummerTotal[0] += getValue(binary, indexes["IsSummer"])
    cpd_IsNotSummerTotal[0] += getNoValue(binary, indexes["IsSummer"])

    cpd_HasPneumoniaTotal[0] += getValue(binary, indexes["HasPneumonia"])
    cpd_NoPneumoniaTotal[0] += getNoValue(binary, indexes["HasPneumonia"])

    # 1 parent
    # expected 4 values
    # 0th index of array is for when the parent is false, 1st index is for when the parent is true

    IsSummerValue = getValue(binary, indexes["IsSummer"])
    cpd_HasFoodPoisoningTotal[IsSummerValue] += getValue(
        binary, indexes["HasFoodPoisoning"]
    )
    cpd_NoFoodPoisoningTotal[IsSummerValue] += getNoValue(
        binary, indexes["HasFoodPoisoning"]
    )

    cpd_HasHayFeverTotal[IsSummerValue] += getValue(binary, indexes["HasHayFever"])
    cpd_NoHayFeverTotal[IsSummerValue] += getNoValue(binary, indexes["HasHayFever"])

    cpd_HasFluTotal[IsSummerValue] += getValue(binary, indexes["HasFlu"])
    cpd_NoFluTotal[IsSummerValue] += getNoValue(binary, indexes["HasFlu"])

    HasFoodPoisoningValue = getValue(binary, indexes["HasFoodPoisoning"])
    cpd_HasGastricProblemsTotal[HasFoodPoisoningValue] += getValue(
        binary, indexes["HasGastricProblems"]
    )
    cpd_NoGastricProblemsTotal[HasFoodPoisoningValue] += getNoValue(
        binary, indexes["HasGastricProblems"]
    )

    HasHayFeverValue = getValue(binary, indexes["HasHayFever"])
    cpd_RashTotal[HasHayFeverValue] += getValue(binary, indexes["Rash"])
    HasPneumoniaValue = getValue(binary, indexes["HasPneumonia"])
    cpd_HasRespiratoryProblemsTotal[HasPneumoniaValue] += getValue(
        binary, indexes["HasRespiratoryProblems"]
    )

    # 2 parents
    # expected 8 values in cpd
    # 0th index when both parents false. 1st index when first parent true, second parent false. 2nd index when first parent false, second parent true. 3rd index when both parents true
    HasFoodPoisoningValue = getValue(binary, indexes["HasFoodPoisoning"])
    HasGastricProblemsValue = getValue(binary, indexes["HasGastricProblems"])
    temp_index = 0 if HasFoodPoisoningValue == 1 else 2
    temp_index += HasGastricProblemsValue
    cpd_VomitsTotal[temp_index] += getValue(binary, indexes["Vomits"])
    cpd_NoVomitTotal[temp_index] += getNoValue(binary, indexes["Vomits"])

    # 3 parents
    # expected 16
    # each index setup us
    # 000, 001, 010, 011, 100, 101, 110, 111
    HasHayFeverValue = HasHayFeverValue
    HasFluValue = getValue(binary, indexes["HasFlu"])
    HasPneumoniaValue = HasPneumoniaValue
    temp_index = 0 if HasHayFeverValue == 1 else 4
    temp_index += 2 if HasFluValue == 1 else 0
    temp_index += HasPneumoniaValue
    cpd_CoughsTotal[temp_index] += getValue(binary, indexes["Coughs"])
    cpd_NoCoughsTotal[temp_index] += getNoValue(binary, indexes["Coughs"])
    cpd_FeverTotal[temp_index] += getValue(binary, indexes["Fever"])
    cpd_NoFeverTotal[temp_index] += getNoValue(binary, indexes["Fever"])

    HasPneumoniaValue = HasPneumoniaValue
    CoughsValue = getValue(binary, indexes["Coughs"])
    FeverValue = getValue(binary, indexes["Fever"])
    temp_index = 0 if HasPneumoniaValue == 1 else 4
    temp_index += 2 if CoughsValue == 1 else 0
    temp_index += FeverValue
    cpd_IsFatiguedTotal[temp_index] += getValue(binary, indexes["IsFatigued"])
    cpd_NotFatiguedTotal[temp_index] += getNoValue(binary, indexes["IsFatigued"])

    # count done

# normalize the counts
tempTotal1 = cpd_IsSummerTotal[0] + cpd_IsNotSummerTotal[0]
new_cpd_IsSummer = [
    [cpd_IsSummerTotal[0] / tempTotal1],
    [cpd_IsNotSummerTotal[0] / tempTotal1],
]

# food poisoning
tempTotal1 = cpd_HasFoodPoisoningTotal[0] + cpd_NoFoodPoisoningTotal[0]
tempTotal2 = cpd_HasFoodPoisoningTotal[1] + cpd_NoFoodPoisoningTotal[1]
"""
[(child false| parent false), (child false| parent true)]
[(child true|parent false), (child true| parent true)]

cpd_NoFoodPoisoningTotal[0] means probability of (child false and parent false)
cpd_NoFoodPoisoningTotal[0]/ tempTotal1 means probability of (child false and parent false) / probability of (parent false)
which gives us probability of (child false | parent false)
"""
new_cpd_HasFoodPoisoning = [
    [
        cpd_NoFoodPoisoningTotal[0] / tempTotal1,
        cpd_NoFoodPoisoningTotal[1] / tempTotal2,
    ],
    [
        cpd_HasFoodPoisoningTotal[0] / tempTotal1,
        cpd_HasFoodPoisoningTotal[1] / tempTotal2,
    ],
]

# hay fever
tempTotal1 = cpd_HasHayFeverTotal[0] + cpd_NoHayFeverTotal[0]
tempTotal2 = cpd_HasHayFeverTotal[1] + cpd_NoHayFeverTotal[1]
new_cpd_HasHayFever = [
    [cpd_NoHayFeverTotal[0] / tempTotal1, cpd_NoHayFeverTotal[1] / tempTotal2],
    [cpd_HasHayFeverTotal[0] / tempTotal1, cpd_HasHayFeverTotal[1] / tempTotal2],
]
# flu
tempTotal1 = cpd_HasFluTotal[0] + cpd_NoFluTotal[0]
tempTotal2 = cpd_HasFluTotal[1] + cpd_NoFluTotal[1]
new_cpd_HasFlu = [
    [cpd_NoFluTotal[0] / tempTotal1, cpd_NoFluTotal[1] / tempTotal2],
    [cpd_HasFluTotal[0] / tempTotal1, cpd_HasFluTotal[1] / tempTotal2],
]
# pneumonia
tempTotal1 = cpd_HasPneumoniaTotal[0] + cpd_NoPneumoniaTotal[0]
new_cpd_HasPneumonia = [
    [cpd_NoPneumoniaTotal[0] / tempTotal1],
    [cpd_HasPneumoniaTotal[0] / tempTotal1],
]

# gastric problems
tempTotal1 = cpd_HasGastricProblemsTotal[0] + cpd_NoGastricProblemsTotal[0]
tempTotal2 = cpd_HasGastricProblemsTotal[1] + cpd_NoGastricProblemsTotal[1]

new_cpd_HasGastricProblems = [
    [
        cpd_NoGastricProblemsTotal[0] / tempTotal1,
        cpd_NoGastricProblemsTotal[1] / tempTotal2,
    ],
    [
        cpd_HasGastricProblemsTotal[0] / tempTotal1,
        cpd_HasGastricProblemsTotal[1] / tempTotal2,
    ],
]

# vomits
tempTotal1 = cpd_VomitsTotal[0] + cpd_NoVomitTotal[0]
tempTotal2 = cpd_VomitsTotal[1] + cpd_NoVomitTotal[1]
tempTotal3 = cpd_VomitsTotal[2] + cpd_NoVomitTotal[2]
tempTotal4 = cpd_VomitsTotal[3] + cpd_NoVomitTotal[3]

"""
[(child false| parent false, false), (child false| parent false, true), (child false| parent true, false), (child false| parent true, true)]
[(child true| parent false, false), (child true| parent false, true), (child true| parent true, false), (child true| parent true, true)]
"""
new_cpd_Vomits = [
    [
        cpd_NoVomitTotal[0] / tempTotal1,
        cpd_NoVomitTotal[1] / tempTotal2,
        cpd_NoVomitTotal[2] / tempTotal3,
        cpd_NoVomitTotal[3] / tempTotal4,
    ],
    [
        cpd_VomitsTotal[0] / tempTotal1,
        cpd_VomitsTotal[1] / tempTotal2,
        cpd_VomitsTotal[2] / tempTotal3,
        cpd_VomitsTotal[3] / tempTotal4,
    ],
]


# rash
tempTotal1 = cpd_RashTotal[0] + cpd_NoRashTotal[0]
tempTotal2 = cpd_RashTotal[1] + cpd_NoRashTotal[1]
new_cpd_Rash = [
    [cpd_NoRashTotal[0] / tempTotal1, cpd_NoRashTotal[1] / tempTotal2],
    [cpd_RashTotal[0] / tempTotal1, cpd_RashTotal[1] / tempTotal2],
]

# coughs
tempTotal1 = cpd_CoughsTotal[0] + cpd_NoCoughsTotal[0]
tempTotal2 = cpd_CoughsTotal[1] + cpd_NoCoughsTotal[1]
tempTotal3 = cpd_CoughsTotal[2] + cpd_NoCoughsTotal[2]
tempTotal4 = cpd_CoughsTotal[3] + cpd_NoCoughsTotal[3]
tempTotal5 = cpd_CoughsTotal[4] + cpd_NoCoughsTotal[4]
tempTotal6 = cpd_CoughsTotal[5] + cpd_NoCoughsTotal[5]
tempTotal7 = cpd_CoughsTotal[6] + cpd_NoCoughsTotal[6]
tempTotal8 = cpd_CoughsTotal[7] + cpd_NoCoughsTotal[7]
new_cpd_Coughs = [
    [
        cpd_NoCoughsTotal[0] / tempTotal1,
        cpd_NoCoughsTotal[1] / tempTotal2,
        cpd_NoCoughsTotal[2] / tempTotal3,
        cpd_NoCoughsTotal[3] / tempTotal4,
        cpd_NoCoughsTotal[4] / tempTotal5,
        cpd_NoCoughsTotal[5] / tempTotal6,
        cpd_NoCoughsTotal[6] / tempTotal7,
        cpd_NoCoughsTotal[7] / tempTotal8,
    ],
    [
        cpd_CoughsTotal[0] / tempTotal1,
        cpd_CoughsTotal[1] / tempTotal2,
        cpd_CoughsTotal[2] / tempTotal3,
        cpd_CoughsTotal[3] / tempTotal4,
        cpd_CoughsTotal[4] / tempTotal5,
        cpd_CoughsTotal[5] / tempTotal6,
        cpd_CoughsTotal[6] / tempTotal7,
        cpd_CoughsTotal[7] / tempTotal8,
    ],
]

# fever
tempTotal1 = cpd_FeverTotal[0] + cpd_NoFeverTotal[0]
tempTotal2 = cpd_FeverTotal[1] + cpd_NoFeverTotal[1]
tempTotal3 = cpd_FeverTotal[2] + cpd_NoFeverTotal[2]
tempTotal4 = cpd_FeverTotal[3] + cpd_NoFeverTotal[3]
tempTotal5 = cpd_FeverTotal[4] + cpd_NoFeverTotal[4]
tempTotal6 = cpd_FeverTotal[5] + cpd_NoFeverTotal[5]
tempTotal7 = cpd_FeverTotal[6] + cpd_NoFeverTotal[6]
tempTotal8 = cpd_FeverTotal[7] + cpd_NoFeverTotal[7]
new_cpd_Fever = [
    [
        cpd_NoFeverTotal[0] / tempTotal1,
        cpd_NoFeverTotal[1] / tempTotal2,
        cpd_NoFeverTotal[2] / tempTotal3,
        cpd_NoFeverTotal[3] / tempTotal4,
        cpd_NoFeverTotal[4] / tempTotal5,
        cpd_NoFeverTotal[5] / tempTotal6,
        cpd_NoFeverTotal[6] / tempTotal7,
        cpd_NoFeverTotal[7] / tempTotal8,
    ],
    [
        cpd_FeverTotal[0] / tempTotal1,
        cpd_FeverTotal[1] / tempTotal2,
        cpd_FeverTotal[2] / tempTotal3,
        cpd_FeverTotal[3] / tempTotal4,
        cpd_FeverTotal[4] / tempTotal5,
        cpd_FeverTotal[5] / tempTotal6,
        cpd_FeverTotal[6] / tempTotal7,
        cpd_FeverTotal[7] / tempTotal8,
    ],
]

# respiratory problems
tempTotal1 = cpd_HasRespiratoryProblemsTotal[0] + cpd_NoRespiratoryProblemsTotal[0]
tempTotal2 = cpd_HasRespiratoryProblemsTotal[1] + cpd_NoRespiratoryProblemsTotal[1]
new_cpd_HasRespiratoryProblems = [
    [
        cpd_NoRespiratoryProblemsTotal[0] / tempTotal1,
        cpd_NoRespiratoryProblemsTotal[1] / tempTotal2,
    ],
    [
        cpd_HasRespiratoryProblemsTotal[0] / tempTotal1,
        cpd_HasRespiratoryProblemsTotal[1] / tempTotal2,
    ],
]
# fatigue
tempTotal1 = cpd_IsFatiguedTotal[0] + cpd_NotFatiguedTotal[0]
tempTotal2 = cpd_IsFatiguedTotal[1] + cpd_NotFatiguedTotal[1]
tempTotal3 = cpd_IsFatiguedTotal[2] + cpd_NotFatiguedTotal[2]
tempTotal4 = cpd_IsFatiguedTotal[3] + cpd_NotFatiguedTotal[3]
tempTotal5 = cpd_IsFatiguedTotal[4] + cpd_NotFatiguedTotal[4]
tempTotal6 = cpd_IsFatiguedTotal[5] + cpd_NotFatiguedTotal[5]
tempTotal7 = cpd_IsFatiguedTotal[6] + cpd_NotFatiguedTotal[6]
tempTotal8 = cpd_IsFatiguedTotal[7] + cpd_NotFatiguedTotal[7]
new_cpd_IsFatigued = [
    [
        cpd_NotFatiguedTotal[0] / tempTotal1,
        cpd_NotFatiguedTotal[1] / tempTotal2,
        cpd_NotFatiguedTotal[2] / tempTotal3,
        cpd_NotFatiguedTotal[3] / tempTotal4,
        cpd_NotFatiguedTotal[4] / tempTotal5,
        cpd_NotFatiguedTotal[5] / tempTotal6,
        cpd_NotFatiguedTotal[6] / tempTotal7,
        cpd_NotFatiguedTotal[7] / tempTotal8,
    ],
    [
        cpd_IsFatiguedTotal[0] / tempTotal1,
        cpd_IsFatiguedTotal[1] / tempTotal2,
        cpd_IsFatiguedTotal[2] / tempTotal3,
        cpd_IsFatiguedTotal[3] / tempTotal4,
        cpd_IsFatiguedTotal[4] / tempTotal5,
        cpd_IsFatiguedTotal[5] / tempTotal6,
        cpd_IsFatiguedTotal[6] / tempTotal7,
        cpd_IsFatiguedTotal[7] / tempTotal8,
    ],
]

# Use calculated CPDs and initiate model.


In [12]:
# make new model and add cpts to it
model = BayesianModel(networkStructure)
cpd_IsSummer = TabularCPD(variable="IsSummer", variable_card=2, values=new_cpd_IsSummer)
cpd_HasFoodPoisoning = TabularCPD(
    variable="HasFoodPoisoning",
    variable_card=2,
    values=new_cpd_HasFoodPoisoning,
    evidence=["IsSummer"],
    evidence_card=[2],
)
cpd_HasHayFever = TabularCPD(
    variable="HasHayFever",
    variable_card=2,
    values=new_cpd_HasHayFever,
    evidence=["IsSummer"],
    evidence_card=[2],
)
cpd_HasFlu = TabularCPD(
    variable="HasFlu",
    variable_card=2,
    values=new_cpd_HasFlu,
    evidence=["IsSummer"],
    evidence_card=[2],
)
cpd_HasPneumonia = TabularCPD(
    variable="HasPneumonia", variable_card=2, values=new_cpd_HasPneumonia
)
cpd_HasGastricProblems = TabularCPD(
    variable="HasGastricProblems",
    variable_card=2,
    values=new_cpd_HasGastricProblems,
    evidence=["HasFoodPoisoning"],
    evidence_card=[2],
)
cpd_Vomits = TabularCPD(
    variable="Vomits",
    variable_card=2,
    values=new_cpd_Vomits,
    evidence=["HasFoodPoisoning", "HasGastricProblems"],
    evidence_card=[2, 2],
)
cpd_Rash = TabularCPD(
    variable="Rash",
    variable_card=2,
    values=new_cpd_Rash,
    evidence=["HasHayFever"],
    evidence_card=[2],
)
cpd_Coughs = TabularCPD(
    variable="Coughs",
    variable_card=2,
    values=new_cpd_Coughs,
    evidence=["HasHayFever", "HasFlu", "HasPneumonia"],
    evidence_card=[2, 2, 2],
)
cpd_Fever = TabularCPD(
    variable="Fever",
    variable_card=2,
    values=new_cpd_Fever,
    evidence=["HasHayFever", "HasFlu", "HasPneumonia"],
    evidence_card=[2, 2, 2],
)
cpd_HasRespiratoryProblems = TabularCPD(
    variable="HasRespiratoryProblems",
    variable_card=2,
    values=new_cpd_HasRespiratoryProblems,
    evidence=["HasPneumonia"],
    evidence_card=[2],
)
cpd_IsFatigued = TabularCPD(
    variable="IsFatigued",
    variable_card=2,
    values=new_cpd_IsFatigued,
    evidence=["HasPneumonia", "Coughs", "Fever"],
    evidence_card=[2, 2, 2],
)

# add cpds to the model
model.add_cpds(
    cpd_IsSummer,
    cpd_HasFoodPoisoning,
    cpd_HasHayFever,
    cpd_HasFlu,
    cpd_HasPneumonia,
    cpd_HasGastricProblems,
    cpd_Vomits,
    cpd_Rash,
    cpd_Coughs,
    cpd_Fever,
    cpd_HasRespiratoryProblems,
    cpd_IsFatigued,
)
# Check if the model is consistent
assert model.check_model()

/usr/local/lib/python3.10/dist-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


# output our cpds. the output from here is used for the report


In [17]:
# printing it directy truncates some of the data
def print_full(cpd):
    backup = TabularCPD._truncate_strtable
    TabularCPD._truncate_strtable = lambda self, x: x
    print(cpd)
    TabularCPD._truncate_strtable = backup


print_full(cpd_IsSummer)
print_full(cpd_HasFoodPoisoning)
print_full(cpd_HasHayFever)
print_full(cpd_HasFlu)
print_full(cpd_HasPneumonia)
print_full(cpd_HasGastricProblems)
print_full(cpd_Vomits)
print_full(cpd_Rash)
print_full(cpd_Coughs)
print_full(cpd_Fever)
print_full(cpd_HasRespiratoryProblems)
print_full(cpd_IsFatigued)

+-------------+-----------+
| IsSummer(0) | 0.0528075 |
+-------------+-----------+
| IsSummer(1) | 0.947192  |
+-------------+-----------+
+---------------------+---------------------+--------------------+
| IsSummer            | IsSummer(0)         | IsSummer(1)        |
+---------------------+---------------------+--------------------+
| HasFoodPoisoning(0) | 0.8080205449261898  | 0.6263598920607868 |
+---------------------+---------------------+--------------------+
| HasFoodPoisoning(1) | 0.19197945507381023 | 0.3736401079392132 |
+---------------------+---------------------+--------------------+
+----------------+---------------------+--------------------+
| IsSummer       | IsSummer(0)         | IsSummer(1)        |
+----------------+---------------------+--------------------+
| HasHayFever(0) | 0.8218350546483424  | 0.6489845192444255 |
+----------------+---------------------+--------------------+
| HasHayFever(1) | 0.17816494535165767 | 0.3510154807555745 |
+----------------+-

# make a function to calculate l1 distance


In [18]:
# check true jpd against my model
def calculate_l1_distance(true_joint_prob, model_joint_prob):
    """Calculates the L1 distance between two joint probability distributions.

    Args:
        true_joint_prob: A pandas Series containing the true joint probabilities.
        model_joint_prob: A pandas Series containing the predicted joint probabilities.

    Returns:
        The L1 distance between the two distributions.
    """

    return np.sum(np.abs(np.array(true_joint_prob) - np.array(model_joint_prob)))

# Read and display the joint probability distribution dataset


In [19]:
# Load the true joint probability distribution

import pandas as pd

df = pd.read_csv("joint.dat", sep="\t", header=None, names=["variable", "probability"])
print(df.head())


# get list of variables, and their probabilities from the jpd
variables = df["variable"].values
true_probabilities = df["probability"].values

   variable  probability
0         0     0.294259
1         1     0.102216
2         2     0.003831
3         3     0.000255
4         4     0.023354


# generate jpd with our model, and also our random model.


In [ ]:
# get model jpd
# generate my model jpd
inference = VariableElimination(model)


# calculate full asignment
probability = inference.query(
    variables=[
        "IsSummer",
        "HasFlu",
        "HasFoodPoisoning",
        "HasHayFever",
        "HasPneumonia",
        "HasRespiratoryProblems",
        "HasGastricProblems",
        "Rash",
        "Coughs",
        "IsFatigued",
        "Vomits",
        "Fever",
    ]
)
generated_jpd = []
for variable in variables:
    bitString = "{0:012b}".format(variable)
    value = probability.get_value(
        IsSummer=int(bitString[0]),
        HasFlu=int(bitString[1]),
        HasFoodPoisoning=int(bitString[2]),
        HasHayFever=int(bitString[3]),
        HasPneumonia=int(bitString[4]),
        HasRespiratoryProblems=int(bitString[5]),
        HasGastricProblems=int(bitString[6]),
        Rash=int(bitString[7]),
        Coughs=int(bitString[8]),
        IsFatigued=int(bitString[9]),
        Vomits=int(bitString[10]),
        Fever=int(bitString[11]),
    )
    generated_jpd.append(value)

random_generated_jpd = []
for variable in variables:
    bitString = "{0:012b}".format(variable)
    value = random_probability.get_value(
        IsSummer=int(bitString[0]),
        HasFlu=int(bitString[1]),
        HasFoodPoisoning=int(bitString[2]),
        HasHayFever=int(bitString[3]),
        HasPneumonia=int(bitString[4]),
        HasRespiratoryProblems=int(bitString[5]),
        HasGastricProblems=int(bitString[6]),
        Rash=int(bitString[7]),
        Coughs=int(bitString[8]),
        IsFatigued=int(bitString[9]),
        Vomits=int(bitString[10]),
        Fever=int(bitString[11]),
    )
    random_generated_jpd.append(value)

# calculate l1 distance among all three JPDs


In [21]:
# calculate_l1_distance(generated_jpd, true_probabilities

randomVsTrue = calculate_l1_distance(random_generated_jpd, true_probabilities)
randomVsCalculated = calculate_l1_distance(random_generated_jpd, generated_jpd)
calculatedVsTrue = calculate_l1_distance(generated_jpd, true_probabilities)


print(
    f"Calculated vs True: {calculatedVsTrue}\nRandom vs True: {randomVsTrue}\nRandom vs Calculated: {randomVsCalculated}"
)

Calculated vs True: 1.97791885044118
Random vs True: 1.9153880436059099
Random vs Calculated: 1.9282288693213765


For unknown reasons ( yet ) the random is perfoming better


# build a function that accepts JPD, evidence and query variables to return the marginal probabilities of our query variables.

find more desc in report and comments


In [28]:
def query(jpd, evidence, variables):
    """
    Infers the marginal probabilities of query variables given evidence from a JPD.

    Args:
        jpd: A list of probabilities for all possible combinations.
        evidence: A dictionary of variables and their evidence values.
        variables: A list of query variables.

    Returns:
        A dictionary of query variables and their marginal probabilities.
    """

    # Get all variables in the JPD
    # we have indexes variable that contain variables and their index in the bitstring
    # find which indexes are in the evidence
    evidence_indexes = [indexes[var] for var in evidence.keys()]
    # iterate through the jpd and get values where the evidence is true
    evidence_values = evidence.values()
    # declare variable for the jpd values that match the evidence
    evidence_jpd = []
    evidence_jpd_indexes = []
    for i, row in enumerate(jpd):
        # get the bitstring
        bitstring = "{0:012b}".format(i)
        # check if the evidence matches the bitstring
        if all(
            bitstring[index] == str(value)
            for index, value in zip(evidence_indexes, evidence_values)
        ):
            evidence_jpd.append(row)
            evidence_jpd_indexes.append(i)
    # now with the evidence jpd we can calculate the marginal probabilities
    marginal_probabilities = {}
    # iterate through the variables and get the marginal probabilities
    for variable in variables:
        # get the index of the variable in the bitstring
        variable_index = indexes[variable]
        # get the marginal probability
        marginal_probability = sum(
            evidence_jpd[i]
            for i in range(len(evidence_jpd))
            if i & (1 << variable_index)
        ) / sum(evidence_jpd)
        marginal_probabilities[variable] = marginal_probability
    return marginal_probabilities

# Make queries and infer knowledge


In [32]:
q1 = query(jpd=generated_jpd, evidence={"Fever": 1, "Coughs": 1}, variables=["HasFlu"])
q2 = query(
    jpd=generated_jpd,
    evidence={"HasPneumonia": 1},
    variables=["Rash", "Coughs", "IsFatigued", "Vomits", "Fever"],
)

trueQ1 = query(
    jpd=true_probabilities, evidence={"Fever": 1, "Coughs": 1}, variables=["HasFlu"]
)
trueQ2 = query(
    jpd=true_probabilities,
    evidence={"HasPneumonia": 1},
    variables=["Rash", "Coughs", "IsFatigued", "Vomits", "Fever"],
)

q3 = query(jpd=generated_jpd, evidence={"IsSummer": 1}, variables=["Vomits"])
trueQ3 = query(jpd=true_probabilities, evidence={"IsSummer": 1}, variables=["Vomits"])

print("My model", q1)
print("True model", trueQ1)
print("My model", q2)
print("True model", trueQ2)
print("My model", q3)
print("True model", trueQ3)

My model {'HasFlu': 0.09916951944662365}
True model {'HasFlu': 0.10000000000378037}
My model {'Rash': 0.3418876761072318, 'Coughs': 0.36404706301302325, 'IsFatigued': 0.19147356806452706, 'Vomits': 0.9471925000000002, 'Fever': 0.0}
True model {'Rash': 0.18730072000067843, 'Coughs': 0.20184436888069573, 'IsFatigued': 0.09735500000058118, 'Vomits': 0.052696000000881545, 'Fever': 0.0}
My model {'Vomits': 0.1970411399895852}
True model {'Vomits': 0.19761928799241965}


In [33]:
# more trials with the function
# coughs when he has pneumonia
q4 = query(jpd=generated_jpd, evidence={"HasPneumonia": 1}, variables=["Coughs"])
trueQ4 = query(
    jpd=true_probabilities, evidence={"HasPneumonia": 1}, variables=["Coughs"]
)
print("coughs when has pneumonia-", q4)
print("true coughs when has pneumonia-", trueQ4)


# vomits and gastric problems when has food poisoning
q5 = query(
    jpd=generated_jpd,
    evidence={"HasFoodPoisoning": 1},
    variables=["Vomits", "HasGastricProblems"],
)
trueQ5 = query(
    jpd=true_probabilities,
    evidence={"HasFoodPoisoning": 1},
    variables=["Vomits", "HasGastricProblems"],
)
print("vomits and gastric problems when has food poisoning-", q5)
print("true vomits and gastric problems when has food poisoning-", trueQ5)

# chance of pnemonia when he's fatigued
q6 = query(jpd=generated_jpd, evidence={"IsFatigued": 1}, variables=["HasPneumonia"])
trueQ6 = query(
    jpd=true_probabilities, evidence={"IsFatigued": 1}, variables=["HasPneumonia"]
)
print("chance of pneumonia when he's fatigued-", q6)
print("true chance of pneumonia when he's fatigued-", trueQ6)

# fever and cough being true should explain away some of the probability of pneumonia even tho he's fatigued
q7 = query(
    jpd=generated_jpd,
    evidence={"Fever": 1, "Coughs": 1, "IsFatigued": 1},
    variables=["HasPneumonia"],
)
trueQ7 = query(
    jpd=true_probabilities,
    evidence={"Fever": 1, "Coughs": 1, "IsFatigued": 1},
    variables=["HasPneumonia"],
)
print("he's fatigued but also has fever and coughs-", q7)
print("true he's fatigued but also has fever and coughs-", trueQ7)

coughs when has pneumonia- {'Coughs': 0.36404706301302325}
true coughs when has pneumonia- {'Coughs': 0.20184436888069573}
vomits and gastric problems when has food poisoning- {'Vomits': 0.9721520756412543, 'HasGastricProblems': 1.0}
true vomits and gastric problems when has food poisoning- {'Vomits': 0.09782844465389545, 'HasGastricProblems': 0.24809644558270758}
chance of pneumonia when he's fatigued- {'HasPneumonia': 0.19384709645055295}
true chance of pneumonia when he's fatigued- {'HasPneumonia': 0.13430880000149528}
he's fatigued but also has fever and coughs- {'HasPneumonia': 0.2018825287271535}
true he's fatigued but also has fever and coughs- {'HasPneumonia': 0.2000000000335995}
